In [ ]:
!pip install opencv-python-headless
!pip install torch torchvision

!pip install mtcnn

import cv2
import os
import glob
import torch
import torchvision
from mtcnn import MTCNN
from PIL import Image
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.0 MB/s eta 0:00:00
Mounted at /gdrive
/gdrive


In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
%cd ..


from pathlib import Path

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15543, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 15543 (delta 60), reused 94 (delta 45), pack-reused 15393
Receiving objects: 100% (15543/15543), 14.59 MiB | 25.15 MiB/s, done.
Resolving deltas: 100% (10584/10584), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
/content


In [ ]:
train_folder = (r'/gdrive/My Drive/bike_dataset/train/')
test_folder = (r'/gdrive/My Drive/bike_dataset/test/')
valid_folder = (r'/gdrive/My Drive/bike_dataset/valid/')

In [ ]:
# Load pre-trained ResNet-50 model
resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))
resnet50.eval()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [ ]:
model_yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model_yolov5.eval()


/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-20 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 14.1M/14.1M [00:00<00:00, 68.6MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [ ]:
def preprocess_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    image_paths = glob.glob(input_folder + '*.jpg')

    for img_path in image_paths:
        img = cv2.imread(img_path)
        resized_img = cv2.resize(img, (224, 224))
        output_path = os.path.join(output_folder, os.path.basename(img_path))
        cv2.imwrite(output_path, resized_img)

preprocessed_train_folder = (r'/gdrive/My Drive/bike_dataset/preprocessed_train/')
preprocessed_test_folder = (r'/gdrive/My Drive/bike_dataset/preprocessed_test/')
preprocessed_valid_folder = (r'/gdrive/My Drive/bike_dataset/preprocessed_valid/')

In [ ]:
preprocess_images(train_folder, preprocessed_train_folder)
preprocess_images(test_folder, preprocessed_test_folder)
preprocess_images(valid_folder, preprocessed_valid_folder)

In [ ]:
# # Person re-identification model (Keypoint R-CNN)
# person_reid_model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True)
# person_reid_model.eval()

# Bike detection model (Faster R-CNN)
bike_detection_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
bike_detection_model.eval()


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 229MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
# def get_person_keypoints(img):
#     transform = torchvision.transforms.Compose([
#         torchvision.transforms.ToTensor()
#     ])
#     input_img = transform(img)
#     input_img = input_img.unsqueeze(0)
#     with torch.no_grad():
#         predictions = person_reid_model(input_img)
#     return predictions

def get_faces(img):
    detector = MTCNN()
    faces = detector.detect_faces(img)
    return faces

def get_bike_and_human_detections_yolov5(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    results = model_yolov5(img)  # Get predictions from YOLOv5
    return results.xyxy[0]  # Get detections in xyxy format


# def get_bike_detections(img):
#     transform = torchvision.transforms.Compose([
#         torchvision.transforms.ToTensor()
#     ])
#     input_img = transform(img)
#     input_img = input_img.unsqueeze(0)
#     with torch.no_grad():
#         predictions = bike_detection_model(input_img)
#     bike_detections = [d for d, c in zip(predictions[0]['boxes'], predictions[0]['labels']) if c == 1] # Assuming bike class ID is 1
#     return bike_detections


In [ ]:
# Import additional required libraries
from IPython.display import display
import matplotlib.pyplot as plt

# Modify the draw_bounding_boxes function to handle YOLOv5 output
def draw_bounding_boxes(img, detections):
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if cls == 0:  # Assuming human class ID is 0
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(img, "Human", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        elif cls == 1:  # Assuming bike class ID is 1
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(img, "Bike", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [ ]:
import numpy as np

threshold = 0.5

def extract_features(img, model):
    preprocess = torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    input_img = preprocess(img)
    input_img = input_img.unsqueeze(0)
    with torch.no_grad():
        features = model(input_img)
    features = features.squeeze().numpy()
    return features

def compare_faces(face1_features, face2_features):
    cosine_similarity = np.dot(face1_features, face2_features) / (np.linalg.norm(face1_features) * np.linalg.norm(face2_features))
    return cosine_similarity

def similarity_score(face1, face2):
    total_score = 0
    keypoints1 = [face1['left_eye'], face1['right_eye'], face1['nose'], face1['mouth_left'], face1['mouth_right']]
    keypoints2 = [face2['left_eye'], face2['right_eye'], face2['nose'], face2['mouth_left'], face2['mouth_right']]

    for p1, p2 in zip(keypoints1, keypoints2):
        x1, y1 = p1
        x2, y2 = p2
        distance = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
        score = 1 - (distance / np.sqrt(2 * (224 ** 2)))
        total_score += score

    return total_score / len(keypoints1)

def is_theft_event(faces, bike_detections):
    if len(faces) != 2 or len(bike_detections) == 0:
        return False

    face1 = faces[0]['keypoints']
    face2 = faces[1]['keypoints']

    if len(face1) == 0 or len(face2) == 0:
        return False

    score = similarity_score(face1, face2)

    if score < threshold:
        return True

    return False

In [ ]:
def save_screenshot(img, output_path):
    cv2.imwrite(output_path, img)

In [ ]:
preprocessed_image_paths = glob.glob(preprocessed_test_folder + '*.jpg')

for img_path in preprocessed_image_paths:
    img = cv2.imread(img_path)
    
    faces = get_faces(img)
    all_detections = get_bike_and_human_detections_yolov5(img)

    draw_bounding_boxes(img, all_detections)

    if is_theft_event(faces, all_detections):
        output_folder = (r'/gdrive/My Drive/bike_dataset/theft/')
    else:
        output_folder = (r'/gdrive/My Drive/bike_dataset/non-theft/')

    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(img_path))
    save_screenshot(img, output_path)

1/1 [==============================] - 0s 95ms/step


1/1 [==============================] - 0s 103ms/step
